In [9]:
import pandas as pd
from get_predict_data import getPredictData
from tqdm import tqdm, tqdm_pandas

In [2]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "esportsTeamId_Blue":"str", "esportsTeamId_Red":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)

In [3]:
df = pd.read_excel("../data/game_ids_for_test.xlsx", index_col=0, dtype=strcolumn_dict)

In [11]:
df = df[df["esportsPlayerId_0"].notna() & df["esportsPlayerId_1"].notna() & df["esportsPlayerId_2"].notna() &\
                                  df["esportsPlayerId_3"].notna() & df["esportsPlayerId_4"].notna() & df["esportsPlayerId_5"].notna() &\
                                    df["esportsPlayerId_6"].notna() & df["esportsPlayerId_7"].notna() & df["esportsPlayerId_8"].notna() &\
                                        df["esportsPlayerId_9"].notna()]

In [12]:
df.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,maxHealth_9,kills_9,deaths_9,assists_9,totalGoldEarned_9,creepScore_9,killParticipation_9,championDamageShare_9,wardsPlaced_9,wardsDestroyed_9
0,2024-04-15 15:00:00,112009683822726475,Play In Groups,EMEA Masters,112009683824430452,1,Macko Esports,MCK,1,BeFive,...,2594,0,3,6,6675,41,0.750000,0.069554,55,12
1,2024-04-15 16:00:00,112009683822726475,Play In Groups,EMEA Masters,112009683824430502,1,Gentle Mates,M8,1,GLORE,...,2813,0,8,12,8464,39,0.800000,0.049700,69,18
2,2024-04-15 17:00:00,112009683822726475,Play In Groups,EMEA Masters,112009683824430432,1,Nigma Galaxy,NGX,1,Anorthosis Famagusta Esports,...,2353,1,2,8,7375,43,0.473684,0.124639,49,23
3,2024-04-15 18:00:00,112009683822726475,Play In Groups,EMEA Masters,112009683824430500,1,Geekay Esports,GK,1,Gentle Mates,...,2222,3,2,11,10105,28,0.736842,0.067231,63,10
4,2024-04-15 19:00:00,112009683822726475,Play In Groups,EMEA Masters,112009683824430428,1,Ruddy Esports,RUD,1,Anorthosis Famagusta Esports,...,2279,1,5,9,6137,25,0.909091,0.077285,43,6


In [13]:
df_sorted = df.sort_values("startTime(match)", ascending=True)
df_sorted.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,maxHealth_9,kills_9,deaths_9,assists_9,totalGoldEarned_9,creepScore_9,killParticipation_9,championDamageShare_9,wardsPlaced_9,wardsDestroyed_9
692,2023-01-16 17:00:00,109467087406180264,Week 1,SuperLiga,109467087407753207,1,Bisons,BSO,1,Barça Esports,...,1751,2,1,9,9361,10,0.846154,0.264044,47,3
693,2023-01-16 18:00:00,109467087406180264,Week 1,SuperLiga,109467087407687599,1,Movistar Riders,MRS,1,FnaticTQ,...,1952,1,2,8,8092,33,1.000000,0.065898,60,21
694,2023-01-16 19:00:00,109467087406180264,Week 1,SuperLiga,109467087407753219,1,Guasones,GSNS,0,Rebels Gaming,...,2240,1,1,18,9380,4,0.904762,0.090933,65,1
695,2023-01-16 20:00:00,109467087406180264,Week 1,SuperLiga,109467087407687619,1,Los Heretics,HRTS,0,Giants,...,2655,0,4,14,8929,42,0.823529,0.037151,63,13
696,2023-01-16 21:00:00,109467087406180264,Week 1,SuperLiga,109467087407687649,1,Finetwork KOI,FNK,0,UCAM Esports,...,2541,0,3,14,7602,33,0.823529,0.084092,34,8


In [14]:
df_sorted.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1050 entries, 692 to 10
Data columns (total 181 columns):
 #    Column                 Dtype         
---   ------                 -----         
 0    startTime(match)       datetime64[ns]
 1    tournamentId           object        
 2    blockName              object        
 3    leagueName             object        
 4    matchId                object        
 5    bestof                 int64         
 6    blueteam_name          object        
 7    blueteam_code          object        
 8    blueteam_win           int64         
 9    redteam_name           object        
 10   redteam_code           object        
 11   redteam_win            int64         
 12   gameNumberInAMatch     int64         
 13   gameId                 object        
 14   patch                  float64       
 15   winner_side            object        
 16   duration               int64         
 17   esportsTeamId_Blue     object        
 18   esportsTeam

In [34]:
def convertRowToApiData(row):
    data = {
        "matchId": row["matchId"],
        "teams" : [
            {
                "esportsTeamId": row["esportsTeamId_Blue"],
                "participantMetadata": [
                    {
                        "esportsPlayerId": row["esportsPlayerId_0"],
                        "role": "top"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_1"],
                        "role": "jungle"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_2"],
                        "role": "mid"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_3"],
                        "role": "bottom"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_4"],
                        "role": "support"
                    }
                ]
            },
            {
                "esportsTeamId": row["esportsTeamId_Red"],
                "participantMetadata": [
                    {
                        "esportsPlayerId": row["esportsPlayerId_5"],
                        "role": "top"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_6"],
                        "role": "jungle"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_7"],
                        "role": "mid"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_8"],
                        "role": "bottom"
                    },
                    {
                        "esportsPlayerId": row["esportsPlayerId_9"],
                        "role": "support"
                    }
                ]

            }
        ]
    }
    winrate = getPredictData(data)
    row["winrate_blue"] = winrate[0][0]
    row["winrate_red"] = winrate[0][1]
    return row
    

In [35]:
tqdm.pandas()
df_sorted_with_winrate = df_sorted.progress_apply(lambda row : convertRowToApiData(row), axis=1)

  0%|          | 0/1050 [00:00<?, ?it/s]

100%|██████████| 1050/1050 [09:57<00:00,  1.76it/s]


In [37]:
df_sorted_with_winrate.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1050 entries, 692 to 10
Data columns (total 183 columns):
 #    Column                 Dtype         
---   ------                 -----         
 0    startTime(match)       datetime64[ns]
 1    tournamentId           object        
 2    blockName              object        
 3    leagueName             object        
 4    matchId                object        
 5    bestof                 int64         
 6    blueteam_name          object        
 7    blueteam_code          object        
 8    blueteam_win           int64         
 9    redteam_name           object        
 10   redteam_code           object        
 11   redteam_win            int64         
 12   gameNumberInAMatch     int64         
 13   gameId                 object        
 14   patch                  float64       
 15   winner_side            object        
 16   duration               int64         
 17   esportsTeamId_Blue     object        
 18   esportsTeam

In [40]:
df_sorted_with_winrate[["startTime(match)", "gameId", "winner_side", "winrate_blue", "winrate_red"]].to_excel("../data/result_of_test_data_of_model_draft_5_7_1.xlsx")